In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.pipeline import make_pipeline
import pickle



from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import F1Score
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.regularizers import L2
from tensorflow import multiply
from tensorflow.keras.callbacks import EarlyStopping
from itertools import product
from tensorflow.random import set_seed


Funciones para búsqueda de hiperparámetros

In [2]:
def regresionLogistica(X, y, n, v_cruzada, seed):
    log_reg = LogisticRegression(max_iter=500)
    parametros_log_reg = {'C': np.logspace(-4, 4, 20), 'penalty': ['l1', 'l2'], 'solver': ['liblinear']}
    busqueda_log_reg = RandomizedSearchCV(estimator=log_reg, param_distributions=parametros_log_reg, n_iter=n, \
                                      scoring='f1_weighted', cv=v_cruzada, verbose=1, random_state=seed, n_jobs=-1)
    
    busqueda_log_reg.fit(X, y)
    parametros = busqueda_log_reg.best_params_
    print(f'Los mejores parámetros son: {parametros} con una puntuación de {busqueda_log_reg.best_score_}')

    modelo = LogisticRegression(**parametros)
    scores = cross_val_score(modelo, X, y, cv=v_cruzada, scoring='f1_weighted')
    return (parametros, scores.mean())


def SGDC(X, y, n, v_cruzada, seed):
    sgd = SGDClassifier()
    parametros_sgdc = {'loss': ['hinge', 'squared_error', 'log_loss'], 'alpha': np.logspace(-5, 0, 10),\
                       'max_iter': [100, 200], 'tol': [1e-3, 1e-4]}
    busqueda_sgdc = RandomizedSearchCV(estimator=sgd, param_distributions=parametros_sgdc, n_iter=n,\
                                       scoring='f1_weighted', cv=v_cruzada, verbose=1, random_state=seed, n_jobs=-1)
    
    busqueda_sgdc.fit(X, y)
    parametros = busqueda_sgdc.best_params_
    print(f'Los mejores parámetros son: {parametros} con una puntuación de {busqueda_sgdc.best_score_}')

    modelo = SGDClassifier(**parametros)
    scores = cross_val_score(modelo, X, y, cv=v_cruzada, scoring='f1_weighted')
    return (parametros, scores.mean())

def bosqueAleatorio(X, y, n, v_cruzada, seed):
    rf = RandomForestClassifier()
    parametros_rf = {'n_estimators': np.arange(100, 200, 20), 'max_features': ['sqrt', 'log2']+list(range(1, 10, 3)), \
                     'max_depth': list(np.arange(5, 20, 5)), 'min_samples_split': np.arange(5, 20, 5)}
    busqueda_rf = RandomizedSearchCV(estimator=rf, param_distributions=parametros_rf, n_iter=n, scoring='f1_weighted',\
                                   cv = v_cruzada, verbose=1, random_state=seed, n_jobs=-1)
    
    busqueda_rf.fit(X, y)
    parametros = busqueda_rf.best_params_
    print(f'Los mejores parámetros son: {parametros} con una puntuación de {busqueda_rf.best_score_}')

    modelo = RandomForestClassifier(**parametros)
    scores = cross_val_score(modelo, X, y, cv=v_cruzada, scoring='f1_weighted')
    return (parametros, scores.mean())

def arbolDecision(X, y, n, v_cruzada, seed):
    dtc = DecisionTreeClassifier()
    parametros_dtc = {'max_depth': list(np.arange(20, 50, 2)), 'min_samples_split': np.arange(10, 20, 2), 'criterion': ['gini', 'entropy']}
    busqueda_dtc = RandomizedSearchCV(estimator=dtc, param_distributions=parametros_dtc, n_iter=n, scoring='f1_weighted',\
                                      cv=v_cruzada, verbose=1, random_state=seed, n_jobs=-1)
    
    busqueda_dtc.fit(X, y)
    parametros = busqueda_dtc.best_params_
    print(f'Los mejores parámetros son: {parametros} con una puntuación de {busqueda_dtc.best_score_}')

    modelo = DecisionTreeClassifier(**parametros)
    scores = cross_val_score(modelo, X, y, cv=v_cruzada, scoring='f1_weighted')
    return (parametros, scores.mean())

def redNeuronal(X, y, n, v_cruzada, seed):
    set_seed(seed)
    def design_model(n_features, lr):
        input_ = Input(shape=(n_features,))
        layer1 = Dropout(0.3)(input_)
        layer2 = Dense(128, kernel_regularizer=L2)(layer1)
        layer3 = Dropout(0.3)(layer2)
        layer4 = Dense(128, kernel_regularizer=L2)(layer3)
        output = Dense(1, activation="sigmoid")(layer4)

        # model
        model = Model(inputs=input_, outputs=output)
        model.compile(loss=BinaryCrossentropy, metrics=[F1Score], optimizer = Adam(learning_rate=lr))
        return model
    
    n_features = X.shape[-1]
    stop = EarlyStopping(monitor='val_loss', mode='min', patience=50, restore_best_weights=True)
    parameter_grid = product((0.1, 0.01, 0.001), (16, 64, 256))

    results = list()
    for comb in parameter_grid:
        model = design_model(n_features, lr=comb[0])
        history = model.fit(X,y,epochs=500, batch_size=comb[1], validation_split=0.2,
                        callbacks = [stop], verbose=0)
        results.append((comb[0], comb[1], history.history['f1_score'][-1]))
    
    best_param = max(sorted(results, key = lambda x: x[2]))
    print(f'Los mejores parámetros son: learning_rate: {best_param[0]}, num_batches: {best_param[1]} con una puntuación de {best_param[2]}')

    model = design_model(n_features, lr=best_param[0])
    scores = np.empty(v_cruzada)
    n_samples = X.shape[0]
    jump = int(n_samples/v_cruzada)
    for i in range(v_cruzada):
        val_idx = range(i*jump,(i+1)*jump)
        model.fit(X.drop(X.iloc[val_idx].index), y.drop(y.iloc[val_idx].index),epochs=500, batch_size=comb[1], validation_data=(X.iloc[val_idx],y.iloc[val_idx]),
                callbacks = [stop], verbose=0)
        scores[i] = stop.best

    return (best_param, scores.mean())


def KNN(X, y, n, v_cruzada, seed):
    knn = KNeighborsClassifier()
    parametros_knn = {'n_neighbors': np.arange(1, 15, 3), 'weights': ['uniform', 'distance'],\
                      'metric': ['euclidean', 'manhattan']}
    busqueda_knn = RandomizedSearchCV(estimator=knn, param_distributions=parametros_knn, n_iter=n,\
                                    scoring='f1_weighted', cv=v_cruzada, verbose=1, random_state=seed, n_jobs=-1)
    
    busqueda_knn.fit(X, y)
    parametros = busqueda_knn.best_params_
    print(f'Los mejores parámetros son: {parametros} con una puntuación de {busqueda_knn.best_score_}')

    modelo = KNeighborsClassifier(**parametros)
    scores = cross_val_score(modelo, X, y, cv=v_cruzada, scoring='f1_weighted')
    return (parametros, scores.mean())



In [ ]:
v_cruzada = 5
seed = 34
n = 30
df = pd.read_csv('df.csv', index_col=0).sample(n=50000, random_state=seed)
df_over = pd.read_csv('df_over.csv', index_col=0).sample(n=50000, random_state=seed)
df = pd.read_csv('df.csv', index_col=0).sample(n=50000, random_state=seed)
df = pd.read_csv('df.csv', index_col=0).sample(n=50000, random_state=seed)

def read_pickle(path):
    with open(path, 'rb') as file:
        columns = pickle.load(file)
    return columns

def hyperparameter_tuning_general(path):
    df_rdx = df[read_pickle(path)]
    X, y = df_rdx.drop('label', axis=1), df_rdx['label']

    return {'LogisticRegression': regresionLogistica(X, y, n, v_cruzada, seed), 
            'SGDClassifier':SGDC(X, y, n, v_cruzada, seed), 
            'RandomForestClassifier':bosqueAleatorio(X, y, n, v_cruzada, seed), 
            'DecisionTreeClassifier':arbolDecision(X, y, n, v_cruzada, seed), 
            'KNeighborsClassifier':KNN(X, y, n, v_cruzada, seed),
            # 'NeuralNetwork': redNeuronal(X, y, n, v_cruzada, seed)
            }

def hyperparameter_tuning_NN(path):
    df_rdx = df[read_pickle(path)]
    X, y = df_rdx.drop('label', axis=1), df_rdx['label']
    return {'NeuralNetwork': redNeuronal(X, y, n, v_cruzada, seed)}

### Búsqueda simultánea: mejores hiperparámetros, mejores modelos, mejor balanceo de dataset

In [ ]:
scores = []
scores.append(hyperparameter_tuning_general('df21_columns.pkl'))
scores.append(hyperparameter_tuning_general('df22_columns.pkl'))
scores.append(hyperparameter_tuning_general('df21_over_columns.pkl'))
scores.append(hyperparameter_tuning_general('df22_over_columns.pkl'))
scores.append(hyperparameter_tuning_general('df21_under_columns.pkl'))
scores.append(hyperparameter_tuning_general('df22_under_columns.pkl'))
scores.append(hyperparameter_tuning_general('df21_mid_columns.pkl'))
scores.append(hyperparameter_tuning_general('df22_mid_columns.pkl'))
with open('scores.pkl', 'wb') as file:
    pickle.dump((scores), file, pickle.HIGHEST_PROTOCOL)

In [ ]:
# mejores hiperparámetros, mejores modelos, mejores balanceos
with open('scores.pkl', 'rb') as file:
    scores = pickle.load(file)

models = ['LogisticRegression', 'SGDClassifier', 'RandomForestClassifier', 'DecisionTreeClassifier', 'KNeighborsClassifier']
param = 0
score = 1

best_params = dict()
model_scores = dict()
score_matrix = np.empty((8, len(models)))

i = 0
for model in models:
    hiperparameter_dataframe = pd.DataFrame([scores[i][model][0] for i in range(len(scores))])
    score_dataframe = pd.DataFrame([scores[i][model][1] for i in range(len(scores))])
    complete_dataframe = pd.concat([hiperparameter_dataframe, score_dataframe], axis=1)

    best_param = hiperparameter_dataframe.mode().iloc[0].to_dict()
    best_params[model] = best_param

    best_score = score_dataframe.max().iloc[0]
    avg_score = score_dataframe.mean().iloc[0]
    model_scores[model] = {'best':best_score, 'average':avg_score}

    score_matrix[:,i] = score_dataframe.values.reshape(1,-1)
    i+= 1

    # print(model)
    # display(complete_dataframe)
    # display(hiperparameter_dataframe.mode())
    # print(best_param)
    # print()

# -------------------------------------------------
# hiperparametro claro excepto para SGDC

# -------------------------------------------------
# model scores visualization: small margin 1.Logistic Regression, 2. RandomForest, 3. SGDClassifier
# pd.DataFrame(model_scores).T.plot.bar()
# plt.ylim((0.8,0.9))
# plt.show()

# -------------------------------------------------
# best balance and filtering (filtering juega un papel, pero no es el objetivo principal)

    # diferencia entre método de filtrado
print(score_matrix[::2].mean()) # variance threshold
print(score_matrix[1::2].mean(), '\n') # k best
    # hay mayor una diferencia del >0.01 entre ambos métodos de filtrado usado

# diferencia entre método de balanceado
for i in range(4): # mse
    print(score_matrix[2*i:2*(i+1)].mean())
    print()
print(score_matrix)
# hay una diferencia >0.01, con el mixto siendo el mejor, aunque la diferencia no sea significativa para escoger un método de balanceado sobresaliente, 
# tampoco importaría en el resultado final si se escoge preliminalmente uno que otro

0.8523182234405609
0.8600814040560797 

0.8545899396164872

0.8563603119490267

0.8562978987983811

0.8575511046293863

[[0.8685737  0.86112893 0.87033156 0.82262689 0.83599927]
 [0.88173491 0.85709983 0.88045592 0.83164272 0.83630565]
 [0.86906003 0.86420349 0.8699391  0.82237719 0.83676863]
 [0.88198528 0.86930574 0.88068736 0.83183509 0.83744121]
 [0.86675366 0.86469121 0.86915533 0.82177005 0.83611171]
 [0.88192675 0.87298375 0.88100598 0.83118532 0.83739522]
 [0.86906003 0.86900508 0.87000993 0.82203004 0.83676863]
 [0.88205688 0.87587536 0.88082719 0.83163001 0.83824791]]


chaos

In [ ]:
hyperparameter_tuning_general('df1_columns.pkl')
# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'solver': 'liblinear', 'penalty': 'l2', 'C': np.float64(1438.44988828766)} con una puntuación de 0.8836523703898143
# 0.8836523703898143
# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'tol': 0.001, 'max_iter': 100, 'loss': 'hinge', 'alpha': np.float64(1e-05)} con una puntuación de 0.8830713771651768
# 0.8705891820233645
# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'n_estimators': np.int64(100), 'min_samples_split': np.int64(15), 'max_features': 'log2', 'max_depth': np.int64(15)} con una puntuación de 0.8822279091007633
# 0.8802700901684221
# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'min_samples_split': np.int64(18), 'max_depth': np.int64(22), 'criterion': 'gini'} con una puntuación de 0.8297647106742028
# 0.8292204390533506

# arreglar tuning, ventaja a quien converge mas rapido
# Los mejores parámetros son: learning_rate: 0.1, num_batches: 256 con una puntuación de 0.4108065068721771
# 0.3101118505001068

# Fitting 5 folds for each of 20 candidates, totalling 100 fits
# Los mejores parámetros son: {'weights': 'uniform', 'n_neighbors': np.int64(13), 'metric': 'manhattan'} con una puntuación de 0.8381402494954733
# 0.8381402494954733

Los mejores parámetros son: learning_rate: 0.1, num_batches: 256 con una puntuación de 0.4108065068721771
0.3101118505001068


In [ ]:
hyperparameter_tuning_general('df1_over_columns.pkl')

# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'solver': 'liblinear', 'penalty': 'l1', 'C': np.float64(11.288378916846883)} con una puntuación de 0.8790116083084459
# 0.8789239112218639
# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'tol': 0.001, 'max_iter': 200, 'loss': 'log_loss', 'alpha': np.float64(1e-05)} con una puntuación de 0.8763173410753795
# 0.870363909928772
# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'n_estimators': np.int64(180), 'min_samples_split': np.int64(10), 'max_features': 'sqrt', 'max_depth': np.int64(15)} con una puntuación de 0.8798856772816105
# 0.8797234595037231
# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'min_samples_split': np.int64(16), 'max_depth': np.int64(20), 'criterion': 'gini'} con una puntuación de 0.8281086733314857
# 0.8277098130939763
# Fitting 5 folds for each of 20 candidates, totalling 100 fits
# Los mejores parámetros son: {'weights': 'distance', 'n_neighbors': np.int64(13), 'metric': 'manhattan'} con una puntuación de 0.8376863481234462
# 0.8376863481234462

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'solver': 'liblinear', 'penalty': 'l1', 'C': np.float64(11.288378916846883)} con una puntuación de 0.8790116083084459
0.8789239112218639
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'tol': 0.001, 'max_iter': 200, 'loss': 'log_loss', 'alpha': np.float64(1e-05)} con una puntuación de 0.8763173410753795
0.870363909928772
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'n_estimators': np.int64(180), 'min_samples_split': np.int64(10), 'max_features': 'sqrt', 'max_depth': np.int64(15)} con una puntuación de 0.8798856772816105
0.8797234595037231
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'min_samples_split': np.int64(16), 'max_depth': np.int64(20), 'criterion': 'gini'} con una puntuación de 0.8281086733314857
0.8277098130939763
Fitting 5 folds for each of 20 candidates

c:\Users\david\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 20 is smaller than n_iter=30. Running 20 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Los mejores parámetros son: {'weights': 'distance', 'n_neighbors': np.int64(13), 'metric': 'manhattan'} con una puntuación de 0.8376863481234462
0.8376863481234462


In [ ]:
hyperparameter_tuning_general('df21_columns.pkl')
# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'solver': 'liblinear', 'penalty': 'l1', 'C': np.float64(1.623776739188721)} con una puntuación de 0.8686068480450551
# 0.8685884377243067
# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'tol': 0.0001, 'max_iter': 100, 'loss': 'hinge', 'alpha': np.float64(0.0001291549665014884)} con una puntuación de 0.869123169398431
# 0.8680290383424178
# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'n_estimators': np.int64(180), 'min_samples_split': np.int64(5), 'max_features': 'sqrt', 'max_depth': np.int64(15)} con una puntuación de 0.870872257158875
# 0.8693060540725103
# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'min_samples_split': np.int64(16), 'max_depth': np.int64(20), 'criterion': 'gini'} con una puntuación de 0.8238308603518425
# 0.8229223637389049
# Fitting 5 folds for each of 20 candidates, totalling 100 fits
# Los mejores parámetros son: {'weights': 'uniform', 'n_neighbors': np.int64(13), 'metric': 'manhattan'} con una puntuación de 0.8359992668647557
# 0.8359992668647557

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'solver': 'liblinear', 'penalty': 'l1', 'C': np.float64(1.623776739188721)} con una puntuación de 0.8686068480450551
0.8685884377243067
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'tol': 0.0001, 'max_iter': 100, 'loss': 'hinge', 'alpha': np.float64(0.0001291549665014884)} con una puntuación de 0.869123169398431
0.8680290383424178
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'n_estimators': np.int64(180), 'min_samples_split': np.int64(5), 'max_features': 'sqrt', 'max_depth': np.int64(15)} con una puntuación de 0.870872257158875
0.8693060540725103
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'min_samples_split': np.int64(16), 'max_depth': np.int64(20), 'criterion': 'gini'} con una puntuación de 0.8238308603518425
0.8229223637389049
Fitting 5 folds for each of 20

c:\Users\david\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 20 is smaller than n_iter=30. Running 20 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Los mejores parámetros son: {'weights': 'uniform', 'n_neighbors': np.int64(13), 'metric': 'manhattan'} con una puntuación de 0.8359992668647557
0.8359992668647557


In [ ]:
hyperparameter_tuning_general('df21_over_columns.pkl')
# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'solver': 'liblinear', 'penalty': 'l1', 'C': np.float64(3792.690190732246)} con una puntuación de 0.8690637259804713
# 0.8689902079249718
# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'tol': 0.001, 'max_iter': 200, 'loss': 'hinge', 'alpha': np.float64(0.0016681005372000592)} con una puntuación de 0.8695568774417272
# 0.8666602968564856
# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'n_estimators': np.int64(180), 'min_samples_split': np.int64(10), 'max_features': 'sqrt', 'max_depth': np.int64(15)} con una puntuación de 0.8706375912843614
# 0.8699943036851534
# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'min_samples_split': np.int64(16), 'max_depth': np.int64(20), 'criterion': 'gini'} con una puntuación de 0.8232128288147627
# 0.8231848480889429
# Fitting 5 folds for each of 20 candidates, totalling 100 fits
# Los mejores parámetros son: {'weights': 'distance', 'n_neighbors': np.int64(13), 'metric': 'manhattan'} con una puntuación de 0.8367686275629154
# 0.8367686275629154

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'solver': 'liblinear', 'penalty': 'l1', 'C': np.float64(3792.690190732246)} con una puntuación de 0.8690637259804713
0.8689902079249718
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'tol': 0.001, 'max_iter': 200, 'loss': 'hinge', 'alpha': np.float64(0.0016681005372000592)} con una puntuación de 0.8695568774417272
0.8666602968564856
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'n_estimators': np.int64(180), 'min_samples_split': np.int64(10), 'max_features': 'sqrt', 'max_depth': np.int64(15)} con una puntuación de 0.8706375912843614
0.8699943036851534
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'min_samples_split': np.int64(16), 'max_depth': np.int64(20), 'criterion': 'gini'} con una puntuación de 0.8232128288147627
0.8231848480889429
Fitting 5 folds for each of 

c:\Users\david\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 20 is smaller than n_iter=30. Running 20 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Los mejores parámetros son: {'weights': 'distance', 'n_neighbors': np.int64(13), 'metric': 'manhattan'} con una puntuación de 0.8367686275629154
0.8367686275629154


In [ ]:
hyperparameter_tuning_general('df22_columns.pkl')
# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'solver': 'liblinear', 'penalty': 'l2', 'C': np.float64(10000.0)} con una puntuación de 0.8817349147911596
# 0.8817349147911596
# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'tol': 0.0001, 'max_iter': 100, 'loss': 'hinge', 'alpha': np.float64(0.0001291549665014884)} con una puntuación de 0.8797487306986795
# 0.877929648312875
# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'n_estimators': np.int64(180), 'min_samples_split': np.int64(10), 'max_features': 'sqrt', 'max_depth': np.int64(15)} con una puntuación de 0.8811952711092832
# 0.8803406313736026
# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'min_samples_split': np.int64(16), 'max_depth': np.int64(20), 'criterion': 'gini'} con una puntuación de 0.8308660422307245
# 0.8308985042919395
# Fitting 5 folds for each of 20 candidates, totalling 100 fits
# Los mejores parámetros son: {'weights': 'distance', 'n_neighbors': np.int64(13), 'metric': 'manhattan'} con una puntuación de 0.8363056535870994
# 0.8363056535870994


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'solver': 'liblinear', 'penalty': 'l2', 'C': np.float64(10000.0)} con una puntuación de 0.8817349147911596
0.8817349147911596
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'tol': 0.0001, 'max_iter': 100, 'loss': 'hinge', 'alpha': np.float64(0.0001291549665014884)} con una puntuación de 0.8797487306986795
0.877929648312875
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'n_estimators': np.int64(180), 'min_samples_split': np.int64(10), 'max_features': 'sqrt', 'max_depth': np.int64(15)} con una puntuación de 0.8811952711092832
0.8803406313736026
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'min_samples_split': np.int64(16), 'max_depth': np.int64(20), 'criterion': 'gini'} con una puntuación de 0.8308660422307245
0.8308985042919395
Fitting 5 folds for each of 20 candida

c:\Users\david\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 20 is smaller than n_iter=30. Running 20 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Los mejores parámetros son: {'weights': 'distance', 'n_neighbors': np.int64(13), 'metric': 'manhattan'} con una puntuación de 0.8363056535870994
0.8363056535870994


In [ ]:
hyperparameter_tuning_general('df22_over_columns.pkl')
# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'solver': 'liblinear', 'penalty': 'l2', 'C': np.float64(3792.690190732246)} con una puntuación de 0.8819852792810661
# 0.8819852792810661
# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'tol': 0.0001, 'max_iter': 100, 'loss': 'hinge', 'alpha': np.float64(0.0001291549665014884)} con una puntuación de 0.8802596768085641
# 0.8802367991149194
# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'n_estimators': np.int64(180), 'min_samples_split': np.int64(5), 'max_features': 'log2', 'max_depth': np.int64(15)} con una puntuación de 0.8807910837927398
# 0.8799058676583137
# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'min_samples_split': np.int64(16), 'max_depth': np.int64(20), 'criterion': 'gini'} con una puntuación de 0.8313389385364051
# 0.8320685712582645
# Fitting 5 folds for each of 20 candidates, totalling 100 fits
# c:\Users\david\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 20 is smaller than n_iter=30. Running 20 iterations. For exhaustive searches, use GridSearchCV.
#   warnings.warn(
# Los mejores parámetros son: {'weights': 'distance', 'n_neighbors': np.int64(13), 'metric': 'manhattan'} con una puntuación de 0.8374412069902707
# 0.8374412069902707



Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'solver': 'liblinear', 'penalty': 'l2', 'C': np.float64(3792.690190732246)} con una puntuación de 0.8819852792810661
0.8819852792810661
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'tol': 0.0001, 'max_iter': 100, 'loss': 'hinge', 'alpha': np.float64(0.0001291549665014884)} con una puntuación de 0.8802596768085641
0.8802367991149194
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'n_estimators': np.int64(180), 'min_samples_split': np.int64(5), 'max_features': 'log2', 'max_depth': np.int64(15)} con una puntuación de 0.8807910837927398
0.8799058676583137
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'min_samples_split': np.int64(16), 'max_depth': np.int64(20), 'criterion': 'gini'} con una puntuación de 0.8313389385364051
0.8320685712582645
Fitting 5 folds for each of 

c:\Users\david\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 20 is smaller than n_iter=30. Running 20 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Los mejores parámetros son: {'weights': 'distance', 'n_neighbors': np.int64(13), 'metric': 'manhattan'} con una puntuación de 0.8374412069902707
0.8374412069902707


In [19]:
def hyperparameter_tuning_specific(df_columns, name, model):
    df_rdx = df[df_columns[name]]
    X, y = df_rdx.drop('label', axis=1), df_rdx['label'] 
    print(name, model(X, y, n, v_cruzada, seed))

In [ ]:
df3_columns = read_pickle('df3_columns.pkl')

hyperparameter_tuning_specific(df3_columns, 'LogisticRegression', regresionLogistica)
hyperparameter_tuning_specific(df3_columns, 'SGDClassifier', SGDC)
hyperparameter_tuning_specific(df3_columns, 'RandomForestClassifier', bosqueAleatorio)
hyperparameter_tuning_specific(df3_columns, 'DecisionTreeClassifier', arbolDecision)
# hyperparameter_tuning_specific(df3_columns, 'NeuralNetwork', redNeuronal)


# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'solver': 'liblinear', 'penalty': 'l2', 'C': np.float64(29.763514416313132)} con una puntuación de 0.8307701271904486
# LogisticRegression 0.8307701271904486
# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'tol': 0.0001, 'max_iter': 200, 'loss': 'hinge', 'alpha': np.float64(3.5938136638046256e-05)} con una puntuación de 0.8355134628645823
# SGDClassifier 0.8363256270804058
# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'n_estimators': np.int64(180), 'min_samples_split': np.int64(15), 'max_features': 'sqrt', 'max_depth': np.int64(5)} con una puntuación de 0.8407902015725431
# RandomForestClassifier 0.8403833332635926
# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'min_samples_split': np.int64(16), 'max_depth': np.int64(20), 'criterion': 'entropy'} con una puntuación de 0.8166143581770171
# DecisionTreeClassifier 0.8166258721117126


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'solver': 'liblinear', 'penalty': 'l2', 'C': np.float64(29.763514416313132)} con una puntuación de 0.8307701271904486
LogisticRegression 0.8307701271904486
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'tol': 0.0001, 'max_iter': 200, 'loss': 'hinge', 'alpha': np.float64(3.5938136638046256e-05)} con una puntuación de 0.8355134628645823
SGDClassifier 0.8363256270804058
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'n_estimators': np.int64(180), 'min_samples_split': np.int64(15), 'max_features': 'sqrt', 'max_depth': np.int64(5)} con una puntuación de 0.8407902015725431
RandomForestClassifier 0.8403833332635926
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'min_samples_split': np.int64(16), 'max_depth': np.int64(20), 'criterion': 'entropy'} con una puntuación de 0.816

In [ ]:
df3_columns = read_pickle('df3_over_columns.pkl')

hyperparameter_tuning_specific(df3_columns, 'LogisticRegression', regresionLogistica)
hyperparameter_tuning_specific(df3_columns, 'SGDClassifier', SGDC)
hyperparameter_tuning_specific(df3_columns, 'RandomForestClassifier', bosqueAleatorio)
hyperparameter_tuning_specific(df3_columns, 'DecisionTreeClassifier', arbolDecision)
# hyperparameter_tuning_specific(df3_columns, 'NeuralNetwork', redNeuronal)

# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'solver': 'liblinear', 'penalty': 'l2', 'C': np.float64(29.763514416313132)} con una puntuación de 0.8307701271904486
# LogisticRegression 0.8307701271904486
# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'tol': 0.001, 'max_iter': 200, 'loss': 'hinge', 'alpha': np.float64(3.5938136638046256e-05)} con una puntuación de 0.8336961748141649
# SGDClassifier 0.8157468825390846
# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'n_estimators': np.int64(180), 'min_samples_split': np.int64(15), 'max_features': 'sqrt', 'max_depth': np.int64(5)} con una puntuación de 0.840917093187246
# RandomForestClassifier 0.8404333706468845
# Fitting 5 folds for each of 30 candidates, totalling 150 fits
# Los mejores parámetros son: {'min_samples_split': np.int64(16), 'max_depth': np.int64(20), 'criterion': 'entropy'} con una puntuación de 0.8165628703939516
# DecisionTreeClassifier 0.8165892306377781



Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'solver': 'liblinear', 'penalty': 'l2', 'C': np.float64(29.763514416313132)} con una puntuación de 0.8307701271904486
LogisticRegression 0.8307701271904486
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'tol': 0.001, 'max_iter': 200, 'loss': 'hinge', 'alpha': np.float64(3.5938136638046256e-05)} con una puntuación de 0.8336961748141649
SGDClassifier 0.8157468825390846
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'n_estimators': np.int64(180), 'min_samples_split': np.int64(15), 'max_features': 'sqrt', 'max_depth': np.int64(5)} con una puntuación de 0.840917093187246
RandomForestClassifier 0.8404333706468845
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'min_samples_split': np.int64(16), 'max_depth': np.int64(20), 'criterion': 'entropy'} con una puntuación de 0.81656